In [1]:
from urllib.parse import urlencode, urlparse
import requests
import time
import json
from pprint import pprint

In [2]:
VERSION = '5.64'
# TOKEN = 'd13e692be69592b09fd22c77a590dd34e186e6d696daa88d6d981e1b4e296b14acb377e82dcbc81dc0f22'
TOKEN = ''

try:
    token_file = "token.json"
    with open(token_file) as tf:
        TOKEN = tf.read()
    print('Токен из файла получен')
except BaseException:
    print('Токен получить не удалось')
# print(TOKEN)

params = {'access_token': TOKEN,
          'v': VERSION, }

Токен из файла получен


In [3]:
# USER_ID = '5030613'
USER_ID = input('Введите id пользователя vk: ')

Введите id пользователя vk: 5030613


Функция получения списка друзей

In [4]:
def get_user_friends(user_id = None):
    params['user_id'] = user_id
    response = requests.get('https://api.vk.com/method/friends.get', params)
    return response.json()['response']['items']

In [5]:
user_list = get_user_friends(USER_ID)
print('Список друзей пользователя получен')

Список друзей пользователя получен


Функция получения групп друзей

In [6]:
def get_user_groups(user_id = None, extended = '0'):
    params['user_id'] = user_id
    params['extended'] = extended
    response = requests.get('https://api.vk.com/method/groups.get', params)
    return response.json()['response']['items']

In [7]:
user_groups = get_user_groups(USER_ID, '1')
print('Список групп пользователя получен')

Список групп пользователя получен


Функция проверки состоит ли пользователь в группе

In [8]:
def is_member(user_id, group_id):
    params['user_id'] = user_id
    params['group_id'] = group_id
    response = requests.get('https://api.vk.com/method/groups.isMember', params)
    return response.json()['response']['member']

In [9]:
def is_member_group (user_list, group_id):
#     str_list = [str(i) for i in user_list]
#     params['user_ids'] = (',').join(str_list)
    ul = [str(i) for i in user_list]
#     pprint(ul)
    str_list = (',').join(ul)
    params['user_ids'] = str_list
    params['group_id'] = group_id
    response = requests.get('https://api.vk.com/method/groups.isMember', params)
    return response.json()['response']

In [10]:
if len(user_list) > 350:
    list_user_list = [user_list[i:i + 350] for i in range(0, len(user_list), 350)]

In [11]:
print('Проверка групп на наличие друзей')
secret_groups = []
for group in user_groups:
    group['friends'] = 0
    group_users_info = []
    if list_user_list:
        for lst in list_user_list:
            group_users_info += is_member_group(lst, group['id'])
    else:
        group_users_info = is_member_group(user_list, group['id'])
    for group_user in group_users_info:
        group['friends'] += group_user['member']
    if group['friends'] == 0:
        print('В группе {} никто не состоит'.format(group['name']))
        secret_groups += group
    else:
        print('Группа {} не секретна'.format(group['name']))
    time.sleep(1)

Проверка групп на наличие друзей
Группа абстрактные мемы для элиты всех сортов не секретна
Группа Образовач не секретна
Группа Graphomaniac не секретна
Группа Интересный Нижний Новгород | Новости не секретна
Группа Анекдоты категории Б не секретна
Группа Ложь пабликов ВК | ЛПВК не секретна
Группа Нижний Новгород. Живая афиша. не секретна
Группа Хабрахабр не секретна
Группа Рик и Морти | Rick and Morty | 3 сезон не секретна
Группа Небес Прозы не секретна
Группа ОНА РАЗВАЛИЛАСЬ не секретна
Группа Пикачу-сальса не секретна
Группа Гарри Поттер и методы рационального мышления не секретна
Группа CMT - Научный подход не секретна
Группа True Russian Wedding (TRW) не секретна
Группа Суровый технарь не секретна
Группа fast food не секретна
Группа Adventure Time Or Die не секретна
Группа Пикачу-еда не секретна
Группа c3e895aa-3b61-45a6-bbb2-cdce55851c68 не секретна
Группа Тинькофф Банк не секретна
Группа Библиотека программиста не секретна
Группа Секира Лектора не секретна
Группа Из Куеды | Блог 

KeyboardInterrupt: 

In [ ]:
with open('groups.json', 'w', encoding='utf8') as outfile:
    json.dump(user_groups, outfile, ensure_ascii=False)